In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.genmod.families import NegativeBinomial, Gamma
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP

from scipy.stats import expon, nbinom, norm, poisson
from scipy.optimize import minimize

import warnings
warnings.filterwarnings("ignore")


# now the DCIS count data is found in an RDA file, which we apparently read using 'pyreadr'
import pyreadr


# to convert Ensemble to Refseq gene names
gene_convert = pyreadr.read_r('/path/to/7_datasets/our_dcis/gene_info/ensemble_to_refseq_gene_name_table.rds')
gene_convert = gene_convert[None]
id_to_name = {gene_id: gene_name for gene_id, gene_name in zip(gene_convert["gene_id"], gene_convert["gene_name"])}


In [ ]:
### Parameters ###

# whether or not we're doing outlier removal using trimmed means
trim_means_flag = True
trim_percent = 10 # 1% usually gets rid of most extreme outliers

# genes must be expressed in this % of patients (between 0-1)
express_percent_limit = 0.2 # set to 0 if you want patient stats (all genes with at least 1 read), set to 0.2 if we want AIC stats of genes with >20% expression

# library adjust (using fractional method)
adjust_for_lib = False

# calculate AIC distance
calc_AIC_dist = False # False saves time when running the full program

# a flag if we want to just do "no ZI" or "NB vs ZINB only"
NB_ZINB_only = False # Only comparing NB to ZINB [trim_percent should be low, maybe even zero]

# trim will remove zeroes, so I don't think we should activate trim when doing NB/ZINB comparison
if (NB_ZINB_only == True):
    trim_percent = 0

no_ZI_AICs = False


In [ ]:
# non-AIC related functions used in this program    

# computation of gene average, fraction of zeroes, and library size
def dataset_stats_generator(df, draw_zero_distribution = True):
    num_genes = df.shape[0]
    num_samples = df.shape[1]
    # Compute the metrics for each row
    row_sums = df.sum(axis = 0)
    fraction_zero_samples = (df == 0).sum(axis=0) / num_genes
    fraction_zero_genes = (df == 0).sum(axis=1) / num_samples
    row_means = df.mean(axis=1)

    if (draw_zero_distribution):
        plt.hist(fraction_zero_genes, bins=100, color='blue', alpha=0.7)
        plt.xlabel('Fraction of Zeroes (Genes)')
        plt.ylabel('Count')
        plt.title("Fraction of Zeroes per Gene")
        plt.show()

        plt.hist(fraction_zero_samples, bins=100, color='blue', alpha=0.7)
        plt.xlabel('Fraction of Zeroes (Samples)')
        plt.ylabel('Count')
        plt.title("Fraction of Zeroes per Sample")
        plt.show()

        plt.hist(row_means, bins=100, color='blue', alpha=0.7)
        plt.xlabel('Means of Gene Expression')
        plt.ylabel('Count')
        plt.title("Distribution of Means of Genes in Dataset")
        plt.show()
      
    # get the average of these 
    avg_library_size = np.round(np.sum(row_sums) / num_samples, decimals = 0)
    avg_zeroes = np.round(np.sum(fraction_zero_samples) / num_samples, decimals = 3)
    avg_mean_expression = np.round(np.mean(row_means), decimals = 3)
    
    # print("Avg Library Size", "Avg Fraction Zeroes", "Avg Mean Expression")
    return avg_library_size, avg_zeroes, avg_mean_expression

# Simulating some data for illustration
#data = np.random.negative_binomial(10, 0.5, 1000)

def fit_to_nb_plot(data, plotrange = 30):

    # Estimating parameters directly from data
    mean = np.mean(data)
    var = np.var(data)
    p = 1 - (mean / var)
    n = mean * (1 - p) / p

    # Plotting
    plt.hist(data, bins=range(plotrange), align='left', density=True, alpha=0.6, color='g')
    plt.plot(bins[:-1], nbinom.pmf(bins[:-1], n, p), 'ro-', lw=2)
    plt.title("Negative Binomial Fit")
    plt.show()

# adjust for library sizes
def library_adjust(data):
    if (adjust_for_lib):
        library_size = data.sum(axis=0)
        
        cleaned_matrix = np.round((data /library_size)*10000000)
        return cleaned_matrix
    else:
        return data

In [ ]:
# functions to compute ZINB

def zinb_loglike(params, counts):
    mu, theta, pi = params
    p = 1 / (1 + mu/theta)
    n = mu * p / (1 - p)
    loglik_pois = nbinom.logpmf(counts, n, p)
    loglik_zero = np.log(pi + (1 - pi) * np.exp(nbinom.logpmf(0, n, p)))
    loglik = np.where(counts == 0, loglik_zero, np.log(1 - pi) + loglik_pois)
    return -np.sum(loglik)

def calculate_aic(loglik, k):
    return 2*k - 2*loglik

def fit_zinb_and_calculate_aic(counts):
    
    initial_params = np.array([np.mean(counts), np.var(counts), 0.5])
    bounds = [(0, None), (0, None), (0, 1)]
    result = minimize(zinb_loglike, initial_params, args=(counts), bounds=bounds)
    mu, theta, pi = result.x
    loglik = -result.fun
    
    k = 3  # Number of parameters
    aic = calculate_aic(loglik, k)
    return mu, theta, pi, aic

In [ ]:
# functions to compute ZIP
def zip_loglike(params, counts):
    mu, pi = params
    loglik_pois = poisson.logpmf(counts, mu)
    loglik_zero = np.log(pi + (1 - pi) * np.exp(poisson.logpmf(0, mu)))
    loglik = np.where(counts == 0, loglik_zero, np.log(1 - pi) + loglik_pois)
    return -np.sum(loglik)

def calculate_aic(loglik, k):
    return 2*k - 2*loglik

def fit_zip_and_calculate_aic(counts):
    
    initial_params = np.array([np.mean(counts), 0.5])
    bounds = [(0, None), (0, 1)]
    result = minimize(zip_loglike, initial_params, args=(counts), bounds=bounds)
    mu, pi = result.x
    loglik = -result.fun
    k = 2  # Number of parameters for ZIP model
    aic = calculate_aic(loglik, k)
    return mu, pi, aic

In [ ]:
# this is the function that computes AIC for all distributions (Gaussian, Exponential, Negative Binomial, ZIP, ZINB)
# and reports which distribution is lowest
# row - a vector of expressions
def manual_aic(row):
    
    row = np.round(row) # it must be count data

    # trimmed mean to remove outliers
    n = len(row)
    
    if (trim_means_flag):
        elements_to_trim = int(np.floor(trim_percent / 100.0 * n))  
        sorted_data = np.sort(row)
        
        if (elements_to_trim > 0):
            row = sorted_data[elements_to_trim:-elements_to_trim]
        else: 
            row = sorted_data

    if (sum(row) <= 0):
        return "ZEROES"

    X = sm.add_constant(np.ones(len(row)))

    # Exponential parameters
    lambda_exp = 1 / np.mean(row)
    log_likelihood_exp = np.sum(expon.logpdf(row, scale=1/lambda_exp))
    aic_exp = 2*1 - 2*log_likelihood_exp  # 1 parameter for exponential

    # log -> linear -> delog
    # using StatsModels to fit to a Gamma (it doesn't have exponential)
    #model_exponential_approx = sm.GLM(row, X, family=Gamma()).fit()
    #print('AIC for Gamma:', model_exponential_approx.aic)
    #aic_exp = model_exponential_approx.aic

    # Compute AIC to NB manually 
    mu_sample = np.mean(row)
    var_sample = np.var(row)
    if (mu_sample == var_sample):
        var_sample = var_sample + 0.0000000000001
    r_estimated = mu_sample**2 / (var_sample - mu_sample)
    
    if (mu_sample + r_estimated) == 0:
        r_estimated = r_estimated + 0.0000000000001
    p_estimated = r_estimated / (mu_sample + r_estimated)
    log_likelihood_nb = np.sum(nbinom.logpmf(row, r_estimated, p_estimated))
    aic_nb = 2*2 - 2*log_likelihood_nb  # 2 parameters for NB
    aic_nb_orig = aic_nb

    # StatsModels method to compute fit to NB
    X = sm.add_constant(np.ones(len(row)))
    model_nb = sm.GLM(row, X, family=NegativeBinomial()).fit()
    aic_nb = model_nb.aic
    #print(model_nb.summary())
    #print("aic GLM", aic_nb)

    res = sm.NegativeBinomial(row, X).fit(start_params=[1,1])
    #print(res.summary())
    #print("aic MLE", res.aic)
    
    aic_nb_orig = res.aic

    # ZINB parameters
    #pi_zinb = np.mean(row == 0)
    #log_likelihood_zeros = np.sum(np.log(pi_zinb) * (row == 0))
    #log_likelihood_non_zeros = np.sum(np.log(1 - pi_zinb) + nbinom.logpmf(row[row != 0], r_estimated, p_estimated))
    #log_likelihood_zinb = log_likelihood_zeros + log_likelihood_non_zeros
    #aic_zinb = 2*3 - 2*log_likelihood_zinb  # 3 parameters for ZINB: pi, r, and p

    #print("Old", aic_zinb)

    mu, theta, pi, aic = fit_zinb_and_calculate_aic(row)
    aic_zinb = aic
    
    #print("New", mu, theta, pi, aic_zinb)
    #print("AIC ZINB", aic_zinb)
    #model_zinb = ZeroInflatedNegativeBinomialP(endog=row, exog=X, exog_infl=X, inflation='logit')
    #results_zinb = model_zinb.fit()

    # print("StatsModels ZINB", results_zinb.aic)


    # Gaussian parameters
    mu_gauss = np.mean(row)
    sigma_gauss = np.std(row)
    log_likelihood_gauss = np.sum(norm.logpdf(row, mu_gauss, sigma_gauss))
    aic_gauss = 2*2 - 2*log_likelihood_gauss  # 2 parameters for Gaussian: mu and sigma
    #print("Manual Gaussian", aic_gauss)

    model_gaussian = sm.GLM(row, X).fit()  # Default is Gaussian
    #print('AIC for Gaussian model:', model_gaussian.aic)
    aic_gauss = model_gaussian.aic

    # alternative method to compute AIC for Gaussian; end result seems ~1% of GLM
    #x = np.random.normal(size=len(row))
    #X = sm.add_constant(x)  # Adds a constant term to the independent variables
    # model = sm.OLS(row, X).fit()


    # Poisson parameters (all methods give the same AIC)
    lambda_pois = np.mean(row)
    log_likelihood_pois = np.sum(poisson.logpmf(row, lambda_pois))
    aic_pois = 2*1 - 2*log_likelihood_pois  # 1 parameter for Poisson: lambda

    #print("Manual Poisson AIC", aic_pois)

    model_poisson = sm.GLM(row, X, family=sm.families.Poisson()).fit()
    aic_pois = model_poisson.aic
    #print("GLM Poisson", aic_pois)

    model_poisson = sm.Poisson(row, X).fit()
    #print("GLM Poisson", model_poisson.aic)


    # ZIP parameters
    #pi_zip = np.mean(row == 0)
    #lambda_zip = np.mean(row[row != 0])
    #log_likelihood_zeros_zip = np.sum(np.log(pi_zip) * (row == 0))
    #log_likelihood_non_zeros_zip = np.sum(np.log(1 - pi_zip) + poisson.logpmf(row[row != 0], lambda_zip))
    #log_likelihood_zip = log_likelihood_zeros_zip + log_likelihood_non_zeros_zip
    #aic_zip = 2*2 - 2*log_likelihood_zip  # 2 parameters for ZIP: pi and lambda

    #print("Old", aic_zip)
    # Usage
    mu, pi, aic = fit_zip_and_calculate_aic(row)
    aic_zip = aic
    #print(f"mu: {mu}, pi: {pi}, AIC: {aic}")


    # sometimes ZIP and ZINB can be NaN if there are no zeroes
    # NB can also become NaN if Mean > Variance (I think)
    # just in case, lets add this check for all of them
    aic_scores = {'aic_zip': aic_zip, 'aic_zinb': aic_zinb, 'aic_nb': aic_nb, 'aic_pois': aic_pois, 'aic_gauss': aic_gauss, 'aic_exp': aic_exp}

    for key in aic_scores:
        if np.isnan(aic_scores[key]) | np.isinf(aic_scores[key]):
            aic_scores[key] = 100000000

    aic_zip, aic_zinb, aic_nb, aic_pois, aic_gauss, aic_exp = aic_scores.values()
        
    # in certain analyses, we might only want to look at certain distributions
    # so we will make the AIC score high for those we don't care about
    if (NB_ZINB_only == True):
        aic_zip, aic_pois, aic_gauss, aic_exp = 10000000, 10000000, 10000000, 10000000
    if (no_ZI_AICs == True):
        aic_zip, aic_zinb = 10000000, 10000000
    
    print(aic_nb_orig, aic_nb, aic_gauss, aic_exp)

    # Compare AICs and determine best fit
    if (aic_nb < aic_pois ) & (aic_nb < aic_gauss) & (aic_nb < aic_exp) & (aic_nb < aic_zip) & (aic_nb < aic_zinb):
        return "NB"
    elif (aic_gauss < aic_nb) & (aic_gauss < aic_pois) & (aic_gauss < aic_exp) & (aic_gauss < aic_zip) & (aic_gauss < aic_zinb):
        return "Gaussian"
    elif (aic_exp < aic_nb) & (aic_exp < aic_pois) & (aic_exp < aic_gauss) & (aic_exp < aic_zip) & (aic_exp < aic_zinb):
        return "Exponential"
    elif (aic_zinb < aic_nb) & (aic_zinb < aic_pois) & (aic_zinb < aic_gauss) & (aic_zinb < aic_zip) & (aic_zinb < aic_exp):
        return "ZINB"
    elif (aic_zip < aic_nb) & (aic_zip < aic_pois) & (aic_zip < aic_gauss) & (aic_zip < aic_zinb) & (aic_zip < aic_exp):
        return "ZIP"
    else:
        return "Poisson"

In [ ]:
# this version of the function computes the same values but instead returns the 
# differential between the NB AIC and other distributions

# row - a vector of expressions
def manual_aic_distfromNB(row):
    
    row = np.round(row) # make it count data
    # New Way - Trimmed Means
    # trimmed mean to remove outliers
    n = len(row)
    
    if (trim_means_flag):
        elements_to_trim = int(np.floor(trim_percent / 100.0 * n))  
        sorted_data = np.sort(row)
        
        if (elements_to_trim > 0):
            row = sorted_data[elements_to_trim:-elements_to_trim]
        else: 
            row = sorted_data

    
    if (sum(row) <= 0):
        return -9999

    # Exponential parameters
    lambda_exp = 1 / np.mean(row)
    log_likelihood_exp = np.sum(expon.logpdf(row, scale=1/lambda_exp))
    aic_exp = 2*1 - 2*log_likelihood_exp  # 1 parameter for exponential
   

    # NB parameters
    mu_sample = np.mean(row)
    var_sample = np.var(row)
    if (mu_sample == var_sample):
        var_sample = var_sample + 0.0000000000001
    r_estimated = mu_sample**2 / (var_sample - mu_sample)
    
    if (mu_sample + r_estimated) == 0:
        r_estimated = r_estimated + 0.0000000000001
    p_estimated = r_estimated / (mu_sample + r_estimated)
    log_likelihood_nb = np.sum(nbinom.logpmf(row, r_estimated, p_estimated))
    aic_nb = 2*2 - 2*log_likelihood_nb  # 2 parameters for NB

    # ZINB parameters
    pi_zinb = np.mean(row == 0)
    log_likelihood_zeros = np.sum(np.log(pi_zinb) * (row == 0))
    log_likelihood_non_zeros = np.sum(np.log(1 - pi_zinb) + nbinom.logpmf(row[row != 0], r_estimated, p_estimated))
    log_likelihood_zinb = log_likelihood_zeros + log_likelihood_non_zeros
    aic_zinb = 2*3 - 2*log_likelihood_zinb  # 3 parameters for ZINB: pi, r, and p

    # Gaussian parameters
    mu_gauss = np.mean(row)
    sigma_gauss = np.std(row)
    log_likelihood_gauss = np.sum(norm.logpdf(row, mu_gauss, sigma_gauss))
    aic_gauss = 2*2 - 2*log_likelihood_gauss  # 2 parameters for Gaussian: mu and sigma

   

    # Poisson parameters
    lambda_pois = np.mean(row)
    log_likelihood_pois = np.sum(poisson.logpmf(row, lambda_pois))
    aic_pois = 2*1 - 2*log_likelihood_pois  # 1 parameter for Poisson: lambda

    # ZIP parameters
    pi_zip = np.mean(row == 0)
    lambda_zip = np.mean(row[row != 0])
    log_likelihood_zeros_zip = np.sum(np.log(pi_zip) * (row == 0))
    log_likelihood_non_zeros_zip = np.sum(np.log(1 - pi_zip) + poisson.logpmf(row[row != 0], lambda_zip))
    log_likelihood_zip = log_likelihood_zeros_zip + log_likelihood_non_zeros_zip
    aic_zip = 2*2 - 2*log_likelihood_zip  # 2 parameters for ZIP: pi and lambda
    
    # ZIP and ZINB can be NaN if there are no zeroes; similarly, NB is NaN if Mean > Variance
    # to account for this, lets set them to a very high AIC if this occurs
    aic_scores = {'aic_zip': aic_zip, 'aic_zinb': aic_zinb, 'aic_nb': aic_nb, 'aic_pois': aic_pois, 'aic_gauss': aic_gauss, 'aic_exp': aic_exp}

    for key in aic_scores:
        if np.isnan(aic_scores[key]):
            aic_scores[key] = 1000000000

    aic_zip, aic_zinb, aic_nb, aic_pois, aic_gauss, aic_exp = aic_scores.values()

    if (NB_ZINB_only == True):
        aic_zip, aic_pois, aic_gauss, aic_exp = 10000000, 10000000, 10000000, 10000000
    if (no_ZI_AICs == True):
        aic_zip, aic_zinb = 10000000, 10000000

    # Compare AICs and determine best fit
    if (aic_nb < aic_pois ) & (aic_nb < aic_gauss) & (aic_nb < aic_exp) & (aic_nb < aic_zip) & (aic_nb < aic_zinb):
        return 0
    elif (aic_gauss < aic_nb) & (aic_gauss < aic_pois) & (aic_gauss < aic_exp) & (aic_gauss < aic_zip) & (aic_gauss < aic_zinb):
        return (aic_nb - aic_gauss)/aic_gauss
    elif (aic_exp < aic_nb) & (aic_exp < aic_pois) & (aic_exp < aic_gauss) & (aic_exp < aic_zip) & (aic_exp < aic_zinb):
        return (aic_nb - aic_exp)/aic_exp
    elif (aic_zinb < aic_nb) & (aic_zinb < aic_pois) & (aic_zinb < aic_gauss) & (aic_zinb < aic_zip) & (aic_zinb < aic_exp):
        return (aic_nb - aic_zinb)/aic_zinb
    elif (aic_zip < aic_nb) & (aic_zip < aic_pois) & (aic_zip < aic_gauss) & (aic_zip < aic_zinb) & (aic_zip < aic_exp):
        return (aic_nb - aic_zip)/aic_zip
    else:
        return (aic_nb - aic_pois)/aic_pois

In [ ]:
# this is a dataset with 528 FFPE breast cancer samples, sequenced from a HiSeq

data = pd.read_csv('/path/to/7_datasets/third_party/GSE167977_third_party_ffpe/GSE167977_Raw_Counts.txt',
                  delimiter='\t')

# filter and compute dispersion
# dispersion of tumours - All Data
tumours_counts = pd.DataFrame(data)
tumours_counts = tumours_counts.drop(tumours_counts.columns[0], axis=1) # column 1
tumours_counts = tumours_counts.drop(tumours_counts.columns[-5:], axis=1) # last 5 columns

# adjust for library size (fraction method)
# should come before the gene filter
tumours_counts_lib_adjust = library_adjust(tumours_counts)

fraction_of_zeroes = (tumours_counts_lib_adjust == 0).mean(axis=1)
filtered_df = tumours_counts_lib_adjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients

print(filtered_df.shape)

dataset_stats = dataset_stats_generator(filtered_df, draw_zero_distribution=True)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])

# print(dataset_stats)

In [ ]:
if(calc_AIC_dist):

    print("GSE167977 - How off NB is to the winning distribution")
    aic_off = filtered_df.apply(manual_aic_distfromNB, axis=1)
    print(aic_off)

    # I want the average, and I want to eliminate -9999 (that's if all genes are zeroes)
    mask = (aic_off != 0) & (aic_off > -9999)

    median_off = np.median(aic_off[mask])
    print("NB AIC is usually: ", median_off)
    count_less_than_10 = np.sum(aic_off[mask] < 0.01)
    count_greater_equal_10 = np.sum(aic_off[mask] >= 0.01)

    print(f"Count of values < 1%: {count_less_than_10}")
    print(f"Count of values >= 1%: {count_greater_equal_10}")


In [ ]:
# function to compute by row
print("GSE167977 - Lowest AIC across all genes")

aic_values = filtered_df.apply(manual_aic, axis=1)
# print(aic_values)    
AIC_top_rank_GSE167977 = aic_values.value_counts()
print(AIC_top_rank_GSE167977)

In [ ]:
data = pd.read_csv('/path/to/7_datasets/third_party/GSE181466_third_party_ffpe/GSE181466_rsem_genes_matrix-97.txt',
                  delimiter='\t')

# patient information splitting is unnecessary, this appears to all be both FFPE and from tumours
# there is subtype and age information in the series matrix file, if we're interested

# dispersion of tumours - All Data
tumours_counts = pd.DataFrame(data)
# removing gene column at position 0
tumours_counts = tumours_counts.drop(tumours_counts.columns[0], axis=1)
# skip genes that are all zeroes, or just one spurrious read somewhere

# adjust for library size (fraction method)
tumours_counts_libadjust = library_adjust(tumours_counts)

fraction_of_zeroes = (tumours_counts_libadjust == 0).mean(axis=1)
filtered_df = tumours_counts_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients
print(filtered_df.shape)

dataset_stats = dataset_stats_generator(filtered_df)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])
print(dataset_stats)

In [ ]:
if (calc_AIC_dist):
    print("GSE181466 - How off NB is to the winning distribution")
    aic_off = filtered_df.apply(manual_aic_distfromNB, axis=1)
    print(aic_off)

    # I want the average, and I want to eliminate -9999 (that's if all genes are zeroes)
    mask = (aic_off != 0) & (aic_off > -9999)

    median_off = np.median(aic_off[mask])
    print("NB AIC is usually: ", median_off)
    count_less_than_10 = np.sum(aic_off[mask] < 0.01)
    count_greater_equal_10 = np.sum(aic_off[mask] >= 0.01)

    print(f"Count of values < 1%: {count_less_than_10}")
    print(f"Count of values >= 1: {count_greater_equal_10}")

In [ ]:
print("GSE181466")

aic_values = filtered_df.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_GSE181466 = aic_values.value_counts()
print(AIC_top_rank_GSE181466)

In [ ]:
## here, we will repeat our plots but for a different data set
all_counts = pyreadr.read_r('/path/to/7_datasets/third_party/GSE146889_third_party_ffpe/GSE146889_GeneCount.rds')
df = all_counts[None] # load all_counts into a pandas data frame

# we need to split the tumors and normals by name
count_TUMOR = df.filter(like='tumor')
count_NORMAL = df.filter(like='normal')

#filtered_tumour = count_TUMOR[count_TUMOR.sum(axis=1) > 1]
#filtered_normal = count_NORMAL[count_NORMAL.sum(axis=1) > 1]

# adjust for library size (fraction method)
count_TUMOR_libadjust = library_adjust(count_TUMOR)

fraction_of_zeroes = (count_TUMOR_libadjust == 0).mean(axis=1)
filtered_tumour = count_TUMOR_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients

print("Tumours (GSE146889)")
dataset_stats = dataset_stats_generator(filtered_tumour)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])
print(count_TUMOR_libadjust.shape)
print(filtered_tumour.shape)

#print(dataset_stats)
# adjust for library size (fraction method)
count_NORMAL_libadjust = library_adjust(count_NORMAL)

fraction_of_zeroes = (count_NORMAL_libadjust == 0).mean(axis=1)
filtered_normal = count_NORMAL_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients

print("Normals")
dataset_stats = dataset_stats_generator(filtered_normal)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])
#print(dataset_stats)

In [ ]:
print("GSE146889 - Tumours")

aic_values = filtered_tumour.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_GSE146889_tumours = aic_values.value_counts()
print(AIC_top_rank_GSE146889_tumours)

In [ ]:
print("GSE146889 - Normal")

aic_values = filtered_normal.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_GSE146889_normals = aic_values.value_counts()
print(AIC_top_rank_GSE146889_normals)

In [ ]:
all_counts = pyreadr.read_r('/path/to/7_datasets/third_party/GSE209998_third_party_ffpe/GSE209998_GeneCount.rds')
sample_information = pyreadr.read_r('/path/to/7_datasets/third_party/GSE209998_third_party_ffpe/GSE209998_Sample_Data.rds')

# now we want to isolate just the expression from a particular type of tissue
df_counts = all_counts[None] # load all_counts into a pandas data frame
df_sample = sample_information[None] # load all_counts into a pandas data frame

# here, we need to match if a sample is normal or tumour by !Sample_source_name_ch1 row

# so I need to: 1) match columns between sample_information and all_counts 
# are they in the same order
columns_df1 = df_counts.columns
columns_df2 = df_sample.columns

# Now we find what samples were tumours and what were normal
samples_row = df_sample.loc["!Sample_source_name_ch1"]

split_dfs = {}
for sample_type in samples_row.unique():
    matching_columns = [col for col in df_counts.columns if col in df_sample.columns and samples_row[col] == sample_type]
    split_dfs[sample_type] = df_counts[matching_columns]

sample_source = df_sample.loc["!Sample_source"]

split_source = {}
for sample_type in sample_source.unique():
    matching_columns = [col for col in df_counts.columns if col in df_sample.columns and sample_source[col] == sample_type]
    split_source[sample_type] = df_counts[matching_columns]


count_FRESH = split_source["Fresh frozen"]
count_FFPE = split_source["FFPE"]

#filtered_ffpe = count_FFPE[count_FFPE.sum(axis=1) > 1]
# adjust for library size (fraction method)
count_FFPE_libadjust = library_adjust(count_FFPE)
fraction_of_zeroes = (count_FFPE_libadjust == 0).mean(axis=1)
filtered_ffpe = np.round(count_FFPE_libadjust[fraction_of_zeroes < (1 - express_percent_limit)]) # must be expressed to this percentage of patients

print("FFPE")
dataset_stats = dataset_stats_generator(filtered_ffpe)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])
# print(dataset_stats)

print("FFPE", filtered_ffpe.shape)


# adjust for library size (fraction method)
count_FRESH_libadjust = library_adjust(count_FRESH)
fraction_of_zeroes = (count_FRESH_libadjust == 0).mean(axis=1)
filtered_fresh = np.round(count_FRESH_libadjust[fraction_of_zeroes < (1 - express_percent_limit)]) # must be expressed to this percentage of patients
print("Fresh", filtered_fresh.shape[1])

dataset_stats = dataset_stats_generator(filtered_fresh)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])

In [ ]:
print("GSE209998 - FFPE Tumours")

aic_values = filtered_ffpe.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_GSE209998_ffpe = aic_values.value_counts()
print(AIC_top_rank_GSE209998_ffpe)

In [ ]:
print("GSE209998 - Fresh/Frozen Tumours")

aic_values = filtered_fresh.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_GSE209998_fresh = aic_values.value_counts()
print(AIC_top_rank_GSE209998_fresh)

In [ ]:
data = pd.read_csv('/path/to/7_datasets/third_party/GSE47462_third_party_ffpe/GSE47462_Raw_counts_Refseq_genes.txt',
                  delimiter='\t')
# Split the DataFrame into subsets based on column names indicating sample type
normal_data = data.filter(like='_normal')
EN_data = data.filter(like='_EN')
DCIS_data = data.filter(like='_DCIS')
IDC_data = data.filter(like='_IDC')

# since there isn't a ton of data, I also want to group tumors
tumours_data = data.loc[:, ~data.columns.str.contains('_normal')]
tumours_data = tumours_data.iloc[:, 1:]

#filtered_tumour = tumours_counts[tumours_counts.sum(axis=1) > 1]
#filtered_normal = normal_counts[normal_counts.sum(axis=1) > 1]

# adjust for library size (fraction method)
tumours_data_libadjust = library_adjust(tumours_data)
fraction_of_zeroes = (tumours_data_libadjust == 0).mean(axis=1)
filtered_tumour = tumours_data_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients

normal_data_libadjust = library_adjust(normal_data)
fraction_of_zeroes = (normal_data_libadjust == 0).mean(axis=1)
filtered_normal = normal_data_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients

print(filtered_tumour.shape)
print(filtered_normal.shape)

print("Tumour")
dataset_stats = dataset_stats_generator(filtered_tumour)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])
# print(dataset_stats)

print("Normal")
dataset_stats = dataset_stats_generator(filtered_normal)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])
# print(dataset_stats)


In [ ]:
print("GSE47462 - Tumours")

aic_values = filtered_tumour.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_GSE47462_tumours = aic_values.value_counts()
print(AIC_top_rank_GSE47462_tumours)

In [ ]:
print("GSE47462 - Normal")
# aic_values = filtered_normal.apply(best_fit, axis=1)
aic_values = filtered_normal.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_GSE47462_normals = aic_values.value_counts()
print(AIC_top_rank_GSE47462_normals)

In [ ]:
# Read the CSV file into a DataFrame
data = pd.read_csv('/path/to/7_datasets/third_party/GSE120795_third_party_ffpe/GSE120795_total_norms_raw_counts.tsv',
                  delimiter='\t')

# in the series matrix"disease: healthy", 
patient_info = pd.read_csv('/path/to/7_datasets/third_party/GSE120795_third_party_ffpe/GSE120795_cell_info.txt',
                  delimiter='\t')

# this filter is present because those filtered out were not FFPE (blood and bone marrow)
mask = patient_info.iloc[0] == "healthy"

filtered_data = patient_info.loc[:, mask]
patient_names = filtered_data.columns
column_names_with_extension = [name + ".fastq.gz" for name in patient_names]
column_names_with_extension = column_names_with_extension[1:]

# Assuming 'second_list' is the list where you want to filter based on column names
filtered_data = data[column_names_with_extension]
ffpe_counts = pd.DataFrame(filtered_data)
#filtered_data = ffpe_counts[ffpe_counts.sum(axis=1) > 1]
print(ffpe_counts.shape)

ffpe_counts_libadjust = library_adjust(ffpe_counts)
fraction_of_zeroes = (ffpe_counts_libadjust == 0).mean(axis=1)
filtered_data = ffpe_counts_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients
print(filtered_data.shape)

dataset_stats = dataset_stats_generator(filtered_data)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])
print(dataset_stats)


In [ ]:
print("GSE120795 - Normal")

aic_values = filtered_data.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_GSE120795_normals = aic_values.value_counts()
print(AIC_top_rank_GSE120795_normals)

In [ ]:
# the GDC Count-Me-In Data
data = pd.read_csv('/path/to/7_datasets/third_party/CountMeIn_BConly_third_party_ffpe/MBC_CMI_Compiled_Counts.tsv',
                  delimiter=' ') # space delimited

tumours_counts = pd.DataFrame(data)
tumours_counts = tumours_counts.drop(tumours_counts.columns[:3], axis=1) # columns 1-3 should be ignored

# library adjust; remove genes expressed < express_percent_limit
tumours_counts_libadjust = library_adjust(tumours_counts)
fraction_of_zeroes = (tumours_counts_libadjust == 0).mean(axis=1)
filtered_df = tumours_counts_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients
print(filtered_df.shape)

dataset_stats = dataset_stats_generator(filtered_df)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])
print(dataset_stats)

In [ ]:
print("Count Me In - Breast Cancer Only")

aic_values = filtered_df.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_CMI = aic_values.value_counts()
print(AIC_top_rank_CMI)

In [ ]:
# Our dataset!
all_counts = pyreadr.read_r('/path/to/7_datasets/dcis/expression_counts.Jan2023_1_2_and_2_2.rds')
vst_norm = pyreadr.read_r('/path/to/7_datasets/dcis/expression_VST_Normalized.Jan2023_1_2_and_2_2.rds')

# this data is loading without issue
ship_data = pyreadr.read_r('/path/to/7_datasets/dcis/ship1_2_full_tbl.Jan2023.With_Stroma_Assignment.rds')

# now we want to isolate just the expression from a particular type of tissue
df = all_counts[None] # load all_counts into a pandas data frame

# Eliminate any samples in the blacklist
ship_df = ship_data[None]
#print(ship_df['blacklist'].value_counts()) # they're all false

# since ship_data already has patients filtered out, lets filter out any patient who isn't on the list
# match by 'sample_name'
df_blacklist_filtered = df[ship_df['sample_name']]

# split the patients by tissue
count_DCIS = df_blacklist_filtered.filter(like='_D')
count_STROMA = df_blacklist_filtered.filter(like='_S')
count_NORMAL = df_blacklist_filtered.filter(like='_N')

# if we want consistency between the 3 sample types
vst_table = vst_norm[None] # we don't apply this anymore because it blocks any gene with >80% frac_zero
filtered_norm_count = count_NORMAL[count_NORMAL.index.isin(vst_table.index)]
filtered_tumour_count = count_DCIS[count_DCIS.index.isin(vst_table.index)]
filtered_stroma_count = count_STROMA[count_STROMA.index.isin(vst_table.index)]

filtered_norm_count_libadjust = library_adjust(filtered_norm_count)
fraction_of_zeroes = (filtered_norm_count_libadjust == 0).mean(axis=1)
filtered_norm_count = filtered_norm_count_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients

filtered_tumour_count_libadjust = library_adjust(filtered_tumour_count)
fraction_of_zeroes = (filtered_tumour_count_libadjust == 0).mean(axis=1)
filtered_tumour_count = filtered_tumour_count_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients

filtered_stroma_count_libadjust = library_adjust(filtered_stroma_count)
fraction_of_zeroes = (filtered_stroma_count_libadjust == 0).mean(axis=1)
filtered_stroma_count = filtered_stroma_count_libadjust[fraction_of_zeroes < (1 - express_percent_limit)] # must be expressed to this percentage of patients



print("DCIS")
print(filtered_tumour_count.shape)
dataset_stats = dataset_stats_generator(filtered_tumour_count)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])

print("Stroma")
print(filtered_stroma_count.shape)
dataset_stats = dataset_stats_generator(filtered_stroma_count)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])

print("Normal")
print(filtered_norm_count.shape)
dataset_stats = dataset_stats_generator(filtered_norm_count)
print("Average Library Size: ", dataset_stats[0])
print("Fraction of Zeroes: ", dataset_stats[1])
print("Average Mean Expression: ", dataset_stats[2])



In [ ]:
print("Our Data: Tumours")
#print(filtered_tumour_count.head(10))

aic_values = filtered_tumour_count.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_DCIS = aic_values.value_counts()
print(AIC_top_rank_DCIS)

In [ ]:
print("Our Data: Normal")

aic_values = filtered_norm_count.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_DCISNorm = aic_values.value_counts()
print(AIC_top_rank_DCISNorm)

In [ ]:
print("Our Data: Stroma")

aic_values = filtered_stroma_count.apply(manual_aic, axis=1)
print(aic_values)    
AIC_top_rank_DCISStrom = aic_values.value_counts()
print(AIC_top_rank_DCISStrom)

In [ ]:
# Creating a heatmap
AIC_top_rank_GSE167977.name = "GSE167977 (BC)"
AIC_top_rank_GSE181466.name = "GSE181466 (TNBC)"
AIC_top_rank_GSE146889_tumours.name = "GSE146889 (Colo./Endo.)"
AIC_top_rank_GSE146889_normals.name = "GSE146889 (Normal)"
AIC_top_rank_GSE209998_ffpe.name = "GSE209998 (BC|FFPE)"
AIC_top_rank_GSE209998_fresh.name = "GSE209998 (BC|FRESH)"
AIC_top_rank_GSE47462_tumours.name = "GSE47462 (BC)"
AIC_top_rank_GSE47462_normals.name = "GSE47462 (Normal)"
AIC_top_rank_GSE120795_normals.name = "GSE120795 (Normal)"
AIC_top_rank_CMI.name = "TMBC Project"
AIC_top_rank_DCIS.name = "DCIS (Tumour)"
AIC_top_rank_DCISNorm.name = "DCIS (Normal)"
AIC_top_rank_DCISStrom.name = "DCIS (Stroma)"


# lets combine the results in one large table
# must adjust for events where one table is missing entries for a particular dist
combined_table = pd.concat([
    AIC_top_rank_GSE47462_tumours, AIC_top_rank_GSE47462_normals,
    AIC_top_rank_GSE120795_normals, 
    AIC_top_rank_GSE146889_tumours, AIC_top_rank_GSE146889_normals,
    AIC_top_rank_GSE167977, AIC_top_rank_GSE181466,
    AIC_top_rank_GSE209998_ffpe, AIC_top_rank_GSE209998_fresh,
    AIC_top_rank_CMI, AIC_top_rank_DCIS, AIC_top_rank_DCISNorm, AIC_top_rank_DCISStrom
    ], axis=1)
# print(combined_table)

# change table to be based on percentages
df_filled = combined_table.fillna(0)
df_percent = df_filled.div(df_filled.sum(axis=0), axis=1) 
df_transposed = df_percent.T

if 'ZEROES' in df_transposed.columns:
    df_transposed = df_transposed.drop('ZEROES', axis=1)

extra = "All"

if (NB_ZINB_only == True):
    new_order = ['NB', 'ZINB']
    extra = "NB_ZINB"
elif (no_ZI_AICs == True):
    new_order = ['NB', 'Exponential', 'Gaussian', 'Poisson']
    extra = "No_ZI"
else:
    new_order = ['NB', 'Exponential', 'Gaussian', 'Poisson', 'ZINB', 'ZIP']

df_transposed = df_transposed[new_order]

df_transposed.rename(columns={'Exponential': 'Gamma'}, inplace=True)

print(df_transposed.round(3))

lib_adj = "No"
if (adjust_for_lib):
    lib_adj = "Yes"

df_transposed.to_csv('/data/lab_vm/refined/preffect/6_dispersion/6.0_Third_Party_Data_Distribution_Evaluation/Percent_Best_AIC_Table.ZeroFract_' + 
                     str(express_percent_limit) + ".Trim_" + str(trim_percent) + ".Lib_Adj_" + str(lib_adj) + "." + str(extra) + ".csv" , index=True)


In [ ]:
# lets use the table to create a heatmap
import seaborn as sns
import matplotlib.pyplot as plt


# Create the heatmap from percentage values above
if (NB_ZINB_only == True):
        # the NB/ZINB plot has just two rows so we should make it narrower
        plt.figure(figsize=(3, 3.7))
elif (no_ZI_AICs == True):
        plt.figure(figsize=(4, 4))
else: 
        plt.figure(figsize=(6, 4))



heatmap = sns.heatmap(df_transposed, annot=True, fmt=".2f", cmap='crest_r', linewidths=.5,
                      annot_kws={"size": 8, "color": 'w'},  # Set annotation text color to black
                      cbar_kws={'shrink': 0.5, 'ticks': [0, 0.5, 1], 'format': '%.2f'})



# make X/Y labels smaller
plt.yticks(fontsize=7)
plt.xticks(fontsize=7, rotation=60)

# draw a horizontal line between certain rows
gap_size = 3
plt.axhline(y=2, color='honeydew', linewidth=gap_size)
plt.axhline(y=3, color='honeydew', linewidth=gap_size)
plt.axhline(y=5, color='honeydew', linewidth=gap_size)
plt.axhline(y=6, color='honeydew', linewidth=gap_size)
plt.axhline(y=7, color='honeydew', linewidth=gap_size)
plt.axhline(y=9, color='honeydew', linewidth=gap_size)
plt.axhline(y=10, color='honeydew', linewidth=gap_size)

# wanted the color bar to display less digits
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=8)  
cbar.set_ticks([cbar.vmin, 0, cbar.vmax])
cbar.set_ticklabels([f'{cbar.vmin:.1f}', '0.0', f'{cbar.vmax:.1f}'])

# Save and display the plot
plt.tight_layout()
plt.savefig('/path/to/6.0_Third_Party_Data_Distribution_Evaluation/Percent_Best_AIC_Table.ZeroFract_' + 
                     str(express_percent_limit) + ".Trim_" + str(trim_percent) + ".Lib_Adj_" + str(lib_adj) + "." + str(extra) +  ".pdf",
             dpi=300, bbox_inches='tight')  # dpi is dots per inch, for resolution

plt.show()

# Optional: Clear the figure after saving, so that future plt calls don't reuse the same figure
plt.clf()